**Problem No. 3**

Use Random Forests to try to get the best possible test accuracy on MNIST. Use Cross Validation to ﬁnd the best settings. How well can you do? You should use the accuracy metric to compare to logistic regression. What are the hyperparameters of your best model? 

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [0]:
mnist = datasets.fetch_openml(name='mnist_784',version='active')

In [0]:
print(mnist.DESCR)

**Author**: Yann LeCun, Corinna Cortes, Christopher J.C. Burges  
**Source**: [MNIST Website](http://yann.lecun.com/exdb/mnist/) - Date unknown  
**Please cite**:  

The MNIST database of handwritten digits with 784 features, raw data available at: http://yann.lecun.com/exdb/mnist/. It can be split in a training set of the first 60,000 examples, and a test set of 10,000 examples  

It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image. It is a good database for people who want to try learning techniques and pattern recognition methods on real-world data while spending minimal efforts on preprocessing and formatting. The original black and white (bilevel) images from NIST were size normalized to fit in a 20x20 pixel box while preserving their aspect ratio. The resulting images contain grey levels as a result of the anti-aliasing technique used by the normalization algorithm. the images were centered in a 28x28 image b

In [0]:
mnist.data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [0]:
mnist.details

{'default_target_attribute': 'class',
 'file_id': '52667',
 'format': 'ARFF',
 'id': '554',
 'licence': 'Public',
 'md5_checksum': '0298d579eb1b86163de7723944c7e495',
 'name': 'mnist_784',
 'processing_date': '2018-10-03 21:23:30',
 'status': 'active',
 'tag': ['AzurePilot',
  'OpenML-CC18',
  'OpenML100',
  'study_1',
  'study_123',
  'study_41',
  'study_99',
  'vision'],
 'upload_date': '2014-09-29T03:28:38',
 'url': 'https://www.openml.org/data/v1/download/52667/mnist_784.arff',
 'version': '1',
 'visibility': 'public'}

In [0]:
np.unique(mnist.target)

array(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'], dtype=object)

In [0]:
mnist.data.shape

(70000, 784)

In [0]:
X = mnist["data"]
y = mnist["target"]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25,shuffle=False) # Splitting the datasets

In [8]:
pip install scikit-optimize

     |████████████████████████████████| 81kB 2.7MB/s 


In [0]:
import skopt
import xgboost as xgb
import lightgbm as lgb
from xgboost import XGBClassifier,plot_importance
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from google.colab import files
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from skopt import BayesSearchCV

In [0]:
def cross_validation(model,X_train,y_train,modelName):
    scoring = 'accuracy'
    score = model_selection.cross_val_score(model, X_train, y_train, cv = 10, scoring=scoring)
    mean_score = score.mean()
    print("Accuracy Value for " + str(modelName) + ": %.3f (%.3f)" % (score.mean(), score.std()))
    return mean_score

In [0]:
 #manually set the hyper parans
random_forest_classifier = RandomForestClassifier(n_estimators=200,max_depth=8,min_samples_split=10,class_weight="balanced",random_state=100)

In [0]:
cross_validation(random_forest_classifier,X_train,y_train,'Random_Forest')

Accuracy Value for Random_Forest: 0.923 (0.005)


0.9229333333333333

In [0]:
## Hyperparameter Tuning of Random Forest Classifier:

ITERATIONS = 20 
TRAINING_SIZE = 100000 
TEST_SIZE = 25000
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = RandomForestClassifier(
        n_jobs = 1
    ),
    search_spaces = {
    # 'min_samples_split': [3, 5, 8, 10, 20], 
    # 'n_estimators' : [100, 500],
    # 'max_depth': [3, 5, 8, 10, 15],
    # 'max_features': [3, 5, 8]

        'n_estimators' : (100, 600),
        'min_samples_split': [3, 5, 8, 10, 20], 
        'max_depth': [3, 5, 8, 10, 15,20],
        'min_samples_leaf':[1,2,5,10,15,20],
        'max_features': ['auto','sqrt','log2']
},   

      #set 2
        # 'n_estimators' : (200, 1000),
        # 'max_depth': [3, 5, 8, 10, 15,20],
        # 'min_child_weight':(1,10),
        # 'learning_rate': (0.01, 1.0, 'log-uniform'),
        # 'colsample_bytree': (0.01, 1.0, 'uniform'),
        # 'colsample_bylevel': (0.2, 1.0, 'uniform'),
        #  'gamma': [0.5, 1, 1.5, 2, 5,8,10],
        #  'max_delta_step':(1,10)

    scoring = 'accuracy',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

In [0]:
def Print_Model_Results(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest accuracy: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")

In [17]:
result = bayes_cv_tuner.fit(X_train, y_train, callback=Print_Model_Results)

#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
#https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769

Model #1
Best accuracy: 0.915
Best params: OrderedDict([('max_depth', 8), ('max_features', 'log2'), ('min_samples_leaf', 20), ('min_samples_split', 5), ('n_estimators', 435)])

Model #2
Best accuracy: 0.9541
Best params: OrderedDict([('max_depth', 20), ('max_features', 'log2'), ('min_samples_leaf', 2), ('min_samples_split', 20), ('n_estimators', 532)])

Model #3
Best accuracy: 0.9541
Best params: OrderedDict([('max_depth', 20), ('max_features', 'log2'), ('min_samples_leaf', 2), ('min_samples_split', 20), ('n_estimators', 532)])

Model #4
Best accuracy: 0.9541
Best params: OrderedDict([('max_depth', 20), ('max_features', 'log2'), ('min_samples_leaf', 2), ('min_samples_split', 20), ('n_estimators', 532)])

Model #5
Best accuracy: 0.9541
Best params: OrderedDict([('max_depth', 20), ('max_features', 'log2'), ('min_samples_leaf', 2), ('min_samples_split', 20), ('n_estimators', 532)])

Model #6
Best accuracy: 0.9589
Best params: OrderedDict([('max_depth', 15), ('max_features', 'log2'), ('min

/usr/local/lib/python3.6/dist-packages/skopt/optimizer/optimizer.py:409: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Model #18
Best accuracy: 0.9637
Best params: OrderedDict([('max_depth', 20), ('max_features', 'auto'), ('min_samples_leaf', 2), ('min_samples_split', 3), ('n_estimators', 600)])

Model #19
Best accuracy: 0.9637
Best params: OrderedDict([('max_depth', 20), ('max_features', 'auto'), ('min_samples_leaf', 2), ('min_samples_split', 3), ('n_estimators', 600)])

Model #20
Best accuracy: 0.9637
Best params: OrderedDict([('max_depth', 20), ('max_features', 'auto'), ('min_samples_leaf', 2), ('min_samples_split', 3), ('n_estimators', 600)])



 How well can you do? You should use the accuracy metric to compare to logistic regression. What are the hyperparameters of your best model? 

The best hyperparameters are : (max_depth=20,max_features='auto',min_samples_leaf=2,min_samples_split=3,n_estimators=600)

In [0]:
# Best params: OrderedDict(max_depth=20,max_features='auto',min_samples_leaf=2,min_samples_split=3,n_estimators=600)


In [0]:
RF_classifier = RandomForestClassifier(max_depth=20,max_features='auto',min_samples_leaf=2,min_samples_split=3,n_estimators=600)  ##The best parameters are mention

In [21]:
RF_classifier.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=600,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
predictions=RF_classifier.predict(X_test)

In [23]:
RF_classifier.score(X_train,y_train)


0.9984571428571428

In [22]:
RF_classifier.score(X_test,y_test)

0.9708571428571429

The Random Forest Classifer gives an accuracy of 0.97085714 after doing hyper parameter tuning

In [0]:
from sklearn.metrics import classification_report

In [0]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1719
           1       0.99      0.99      0.99      1929
           2       0.96      0.97      0.97      1787
           3       0.97      0.97      0.97      1790
           4       0.98      0.96      0.97      1735
           5       0.98      0.96      0.97      1574
           6       0.98      0.99      0.98      1672
           7       0.97      0.96      0.97      1844
           8       0.96      0.96      0.96      1721
           9       0.94      0.95      0.95      1729

    accuracy                           0.97     17500
   macro avg       0.97      0.97      0.97     17500
weighted avg       0.97      0.97      0.97     17500



In [0]:
#https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769

#https://medium.com/@gabrielziegler3/multiclass-multilabel-classification-with-xgboost-66195e4d9f2d

ITERATIONS = 20

# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = xgb.XGBClassifier(
        n_jobs = 1,
        objective = 'multi:softmax',
        eval_metric = 'mlogloss',
        silent=1,
        num_class=10,
        tree_method='approx'
    ),
      search_spaces = {
        # 'min_samples_split': [3, 5, 8, 10, 20], 
        # 'n_estimators' : [100, 500],
        # 'max_depth': (3,20),
        # 'max_features': [3, 5, 8],
        # 'learning_rate': (0.01, 1.0, 'log-uniform'),
        # 'colsample_bytree': (0.1, 1.0, 'uniform'),
        # 'colsample_bylevel': (0.4, 1.0, 'uniform'),

        
        # 'n_estimators' : (200, 1000),
        # 'max_depth': [3, 5, 8, 10, 15,20],
        # 'min_child_weight':(1,10),
        # 'learning_rate': (0.01, 1.0, 'log-uniform'),
        # 'colsample_bytree': (0.01, 1.0, 'uniform'),
        # 'colsample_bylevel': (0.2, 1.0, 'uniform'),
        #  'gamma': [0.5, 1, 1.5, 2, 5,8,10],
        #  'max_delta_step':(1,10)
  

        'min_samples_split': [3, 5, 8, 10, 20], 
        'n_estimators' : [100, 500],
        'max_depth': [3, 5, 8, 10, 15],
        'max_features': [3, 5, 8],
        'learning_rate': (0.01, 1.0, 'log-uniform')
        # 'colsample_bytree': (0.01, 1.0, 'uniform'),
        # 'colsample_bylevel': (0.01, 1.0, 'uniform') 

    },     
    scoring = 'accuracy',
    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = ITERATIONS,   
    verbose = 0,
    refit = True,
    random_state = 42
)

In [0]:
def Print_Model_Results(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest accuracy: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))
    
    # Save all model results
    clf_name = bayes_cv_tuner.estimator.__class__.__name__
    all_models.to_csv(clf_name+"_cv_results.csv")
    

In [0]:
result = bayes_cv_tuner.fit(X_train, y_train, callback=Print_Model_Results)

#https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
#https://www.kaggle.com/nanomathias/bayesian-optimization-of-xgboost-lb-0-9769

The best hyperparameters for XGBoost Algorithm are max_depth=15,max_features=5,min_samples_split=3,n_estimators=402,colsample_bylevel=0.8142720284737898,colsample_bytree = 0.1801528457825951,learning_rate = 0.1570703295827246)  

 Use Gradient Boosting to do the same. Try your best to tune your hyper parameters. What are the hyperparameters of your best model?


In [0]:
XGB_classifier = XGBClassifier(max_depth=15,max_features=5,min_samples_split=3,n_estimators=402,colsample_bylevel=0.8142720284737898,colsample_bytree = 0.1801528457825951,learning_rate = 0.1570703295827246) 

In [0]:
XGB_classifier.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree',
              colsample_bylevel=0.8142720284737898, colsample_bynode=1,
              colsample_bytree=0.1801528457825951, gamma=0,
              learning_rate=0.1570703295827246, max_delta_step=0, max_depth=15,
              max_features=5, min_child_weight=1, min_samples_split=3,
              missing=None, n_estimators=402, n_jobs=1, nthread=None,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1, verbosity=1)

In [0]:
xgb_predictions = XGB_classifier.predict(X_test)

In [0]:
XGB_classifier.score(X_train,y_train) # Training Accuracy is 1.0

1.0

The cross validation accuracy for XGBoost is 0.97619

In [0]:
cross_validation(XGB_classifier,X_train,y_train,'XGB')

Accuracy Value for XGB: 0.976 (0.003)


0.976190476190476

In [0]:
XGB_classifier.score(X_test,y_test)

0.9796

The Testing Accuracy is 0.9796